In [53]:
import os,tables,numpy as np,matplotlib.pyplot as plt,pandas as pd,json,h5py
from scipy.io import loadmat
from collections import Counter
from  itertools import combinations
from dataholder import Data
import random

In [5]:
os.getcwd()

'/media/mhealthra2/Data/heart_sound/Adversarial-Heart-Sound-Classification/codes'

In [165]:
fold_dir = '../../feature/potes_1DCNN/balancedCV/folds/folds_phys_compare_pascal/'
fold_dir = '../../feature/potes_1DCNN/balancedCV/folds/all_folds_wav_name/'
fold_dir = '../../feature/potes_1DCNN/balancedCV/folds/individual_fold_beats_repeated/'
os.listdir(fold_dir)

['fold_a.mat', 'fold_b.mat', 'fold_c.mat', 'fold_d.mat']

In [7]:
for x in os.listdir(fold_dir):
    data = h5py.File(fold_dir+x, 'r')
    print(x,list(data.keys()))

compare.mat ['#refs#', 'testX', 'test_parts', 'trainX', 'trainY', 'train_parts', 'valX', 'valY', 'val_parts', 'val_wav_name', 'wav_name']
fold_a.mat ['#refs#', 'trainX', 'trainY', 'train_files', 'train_parts', 'wav_name']
fold_b.mat ['#refs#', 'trainX', 'trainY', 'train_files', 'train_parts', 'wav_name']
fold_c.mat ['#refs#', 'trainX', 'trainY', 'train_files', 'train_parts', 'wav_name']
fold_d.mat ['#refs#', 'trainX', 'trainY', 'train_files', 'train_parts', 'wav_name']
fold_e.mat ['#refs#', 'trainX', 'trainY', 'train_files', 'train_parts', 'wav_name']
fold_f.mat ['#refs#', 'trainX', 'trainY', 'train_files', 'train_parts', 'wav_name']
pascalA.mat ['#refs#', '#subsystem#', 'file_name', 'trainX', 'trainY', 'train_parts', 'wav_name']
pascalB.mat ['#refs#', '#subsystem#', 'file_name', 'trainX', 'trainY', 'train_parts', 'wav_name']


In [8]:
[str(x) for x in data['file_name'][0]]

['3707764736', '2', '1', '1', '1', '1']

In [193]:
import matplotlib.pyplot as plt,h5py,numpy as np
from collections import Counter
class Data():
    def __init__(self,path,f,n,severe = True,split=0,normalize=False,shuffle=None):
        
        if(split>=1 or split<0):
            print("make sure split follow  1<split<=0 ")
            raise ValueError
        self.split = split
        self.data = h5py.File(path+f, 'r')
        self.file = f
        self.dom = n
        self.trainX = np.array(self.data['trainX'][:]).astype('float32')
        self.trainY = self.data['trainY'][:][0].astype('int8')
        self.train_parts = self.data['train_parts'][0].astype('int32')
        self.wav_name = [''.join([chr(c[0]) for c in self.data[stp]]) for stp in self.data['wav_name'][0]]
        self.valX = None
        self.valY = None
        self.val_wav_name = None
        self.valdomY = None
        self.val_parts = None
        self.domainY = [n]*self.trainY.shape[0]
        
        ##calculate the normal and abnormal beats count
        self.normal = Counter(self.trainY)[0]
        self.abnormal = Counter(self.trainY)[1]
        self.total = self.normal+self.abnormal
        self.normfiles, self.abnormfiles = self.parts() 
        
        if(f[:3]=='com'):self.processCompare(severe) ### select severe files only
        else:
            self.trainY[self.trainY<0] = 0   
        if('fold_e' in f):self.processE()
        if(shuffle is not None):self.shuffle_data(shuffle)
        if(split>0):self.split_data(split)
        if(normalize):self.normalize_data()
            
        
    def shuffle_data(self,seed=0):
        ## The shuffle parameter is None in init(). but if provided and int value it will be
        ## used as the random seed
        if(self.trainX is not None):
            xx = self.trainX.copy()
            yy = self.trainY.copy()
            pr = [x for x in range(len(self.train_parts))]
            random.Random(seed).shuffle(pr)
            s = 0
            for i,x in enumerate(pr):
                xx[:,s:s+self.train_parts[x]] = self.trainX[:,sum(self.train_parts[:x]):sum(self.train_parts[:x])+self.train_parts[x]]
                yy[s:s+self.train_parts[x]] = self.trainY[sum(self.train_parts[:x]):sum(self.train_parts[:x])+self.train_parts[x]]
                s = s+self.train_parts[x]
            self.train_parts = self.train_parts[pr]
            self.wav_name = [self.wav_name[pr[x]] for x in range(len(self.wav_name))]
        if(self.valX is not None):
            xx = self.valX.copy()
            yy = self.valY.copy()
            pr = [x for x in range(len(self.val_parts))]
            random.Random(seed).shuffle(pr)
            s = 0
            for i,x in enumerate(pr):
                xx[:,s:s+self.train_parts[x]] = self.valX[:,sum(self.train_parts[:x]):sum(self.train_parts[:x])+self.train_parts[x]]
                yy[s:s+self.train_parts[x]] = self.valY[sum(self.train_parts[:x]):sum(self.train_parts[:x])+self.train_parts[x]]
                s = s+self.train_parts[x]
            self.train_parts = self.val_parts[pr]
            self.val_wav_name = [self.val_wav_name[pr[x]] for x in range(len(self.val_wav_name))]
    def normalize_data(self):
        self.trainX = np.array([x/(max(abs(x)+10e-6)) for x in self.trainX.transpose()]).transpose()
    def split_data(self,split):
        if(self.file[:3]=='com'):
            self.valX = self.data['valX'][:]
            self.valY = self.data['valY'][:][0]
            self.valY[self.valY>0] = 1
            self.val_parts = self.data['val_parts'][0].astype('int32')
            self.valdomY = [self.dom]*self.valY.shape[0]
            return
        taken = 0
        left = 0
        tmpX = None
        tmpY = None
        parts = []
        wav_name = []
        self.val_wav_name = []
        
        self.val_parts = []
        for j,x in enumerate(self.train_parts):
            if(taken<split*self.total):
                taken = taken + x
                if(self.valX is None):
                    self.valX = self.trainX[:,left:x+left]
                    self.valY = self.trainY[left:x+left]
                else:
                    
                    self.valX = np.concatenate((self.valX,self.trainX[:,left:x+left]),axis=1)
                    self.valY = np.concatenate((self.valY,self.trainY[left:x+left]),axis=0)
                self.val_parts.append(x)
                self.val_wav_name.append(self.wav_name[j])
            else:
                if(tmpX is None):
                    tmpX = self.trainX[:,left:x+left]
                    tmpY = self.trainY[left:x+left]
                else:
                    tmpX = np.concatenate((tmpX,self.trainX[:,left:x+left]),axis=1)
                    tmpY = np.concatenate((tmpY,self.trainY[left:x+left]),axis=0)
                parts.append(x)
                wav_name.append(self.wav_name[j])
            left = left + x
        self.trainX = tmpX
        self.trainY = tmpY
        self.train_parts = parts
        self.wav_name = wav_name
        self.domainY = [self.dom]*self.trainY.shape[0]
        self.valdomY = [self.dom]*self.valY.shape[0]
        del tmpX,tmpY,parts
    def processCompare(self,severe):
        if(severe):
            print("fixed implementation, normal = 0, mild = 1, sever = 2. mild is being selected")
            self.sevX = None
            self.sevY = self.trainY[self.trainY%2==0]
            self.sev_parts = []
            self.sev_wav_name = []
            left = int(0)
            for j,x in enumerate(self.train_parts):
                x = int(x)
                if(all([i%2==0 for i in self.trainY[left:x+left]])):
                    if(self.sevX is None):
                        self.sevX = self.trainX[:,left:x+left]
                    else: 
                        self.sevX = np.concatenate((self.sevX,self.trainX[:,left:x+left]),axis=1)
                    self.sev_parts.append(x)
                    self.sev_wav_name.append(self.wav_name[j])
                left = x + left
                
            self.trainX = self.sevX
            self.trainY = self.sevY
            self.train_parts = self.sev_parts
            self.domainY = [self.dom]*self.trainY.shape[0]
            self.normal = Counter(self.trainY)[0]
            self.abnormal = Counter(self.trainY)[1]
            self.total = self.normal+self.abnormal
            self.normfiles, self.abnormfiles = self.parts()
        else:
            self.trainY[self.trainY>0] = 1
    def processE(self):
        self.trainY[self.trainY<0] = 0
        nn = ab = 0
        idx = []
        left = 0
        parts = []
        wav_name = []
        for j,x in enumerate(self.train_parts):
            if(all([(l == 0) for l in self.trainY[left:left+x]])):
                if(nn<self.abnormal):
                    idx = idx + [True]*x
                    nn = nn + x
                    parts.append(x)
                    wav_name.append(self.wav_name[j])
                else:idx = idx + [False]*x
            else:
                if(ab<self.abnormal):
                    idx = idx + [True]*x
                    ab = ab + x
                    parts.append(x)
                    wav_name.append(self.wav_name[j])
                else:idx = idx + [False]*x
            left = left + x
        self.trainY = self.trainY[idx]
        self.trainX = np.transpose(np.transpose(self.trainX)[idx])
        self.train_parts = parts
        self.wav_name = wav_name
        self.total = nn+ab
        self.domainY = self.domainY[:self.total]
    def parts(self):
        y = 0
        nn = 0
        ab = 0
        for x in self.train_parts:
            if(sum(self.trainY[y:y+int(x)])>0):
                ab = ab + 1
            else: nn = nn +1 
            y = int(x)+y
        #print(nn+ab,nn,ab)
        return nn,ab
    def pie(self):
        colors = ['gold','lightskyblue']
        explode = (0.07, 0)
        size = [self.normal,self.abnormal]
        labels = ['N', 'Ab']
        plt.pie(size,labels=labels,colors = colors,startangle=140,explode=explode,
               shadow=True,autopct=self.value)
        plt.title(self.dom)
    def value(self,val):
        return int(self.total*val/100)
    def details(self):
        if(self.trainX is  None):print("TrainX None/ ",end='')
        if(self.trainY is  None):print("trainY None/ ",end='')
        if(self.domainY is None):print("Dom Y/ ",end='')
        if(self.train_parts is  None):print("train parts/ ")
        if(self.valX is None):print("valX/ ",end='')
        if(self.valY is None) :print("valY/ ",end='')
        if(self.valdomY is None):print("Valdom/ ",end='')
        if(self.val_parts is  None):print("val parts/ ")

In [206]:
d = Data(fold_dir,'fold_a.mat','a',shuffle = 1,split=0.1)

In [199]:
d.trainX.shape, d.trainY.shape, sum(d.train_parts),d.valX.shape

((2500, 13899), (13899,), 13899, (2500, 1093))

In [205]:
s0 = d.val_wav_name

In [207]:
s1 = d.val_wav_name

In [48]:
dd = Data(fold_dir,'compare.mat','i',True)
dd.details()

['trainX', 'trainY', 'train_files', 'train_parts', 'valX', 'valY', 'val_files', 'val_parts']
val parts None


In [213]:
s0nots1 = [x for x in s0 if x not in s1]
s0s1 = [x for x in s0 if x in s1]

In [214]:
s0,s1,s0s1

(['a0232.wav',
  'a0168.wav',
  'a0385.wav',
  'a0264.wav',
  'a0308.wav',
  'a0181.wav',
  'a0122.wav',
  'a0126.wav',
  'a0007.wav',
  'a0326.wav',
  'a0382.wav',
  'a0237.wav',
  'a0054.wav',
  'a0256.wav',
  'a0368.wav',
  'a0178.wav',
  'a0161.wav',
  'a0119.wav',
  'a0248.wav',
  'a0377.wav',
  'a0093.wav',
  'a0037.wav',
  'a0029.wav',
  'a0116.wav',
  'a0344.wav',
  'a0071.wav',
  'a0082.wav'],
 ['a0387.wav',
  'a0032.wav',
  'a0132.wav',
  'a0175.wav',
  'a0139.wav',
  'a0046.wav',
  'a0290.wav',
  'a0201.wav',
  'a0155.wav',
  'a0295.wav',
  'a0165.wav',
  'a0327.wav',
  'a0143.wav',
  'a0233.wav',
  'a0375.wav',
  'a0316.wav',
  'a0124.wav',
  'a0024.wav',
  'a0068.wav',
  'a0224.wav',
  'a0104.wav',
  'a0060.wav',
  'a0101.wav',
  'a0341.wav',
  'a0004.wav',
  'a0319.wav',
  'a0182.wav',
  'a0388.wav',
  'a0237.wav'],
 ['a0237.wav'])

In [148]:
pp,pp[pr],pr

(array([3, 4, 1, 5]), array([1, 3, 4, 5]), [2, 0, 1, 3])

In [8]:
x = np.random.randint(0,10,(2500,7000))
#p = np.random.randint(20,40,(2500,7000))

In [152]:
xx = np.random.randint(0,10,(5,13))
yy = np.random.randint(0,30,(13))
xxx = xx.copy()
yyy = yy.copy()
xx,xxx,yy,yyy

(array([[2, 1, 3, 5, 5, 6, 7, 7, 8, 4, 1, 9, 8],
        [5, 9, 5, 2, 0, 9, 2, 1, 9, 7, 9, 7, 6],
        [1, 7, 4, 0, 0, 7, 2, 2, 4, 1, 6, 4, 0],
        [6, 8, 1, 8, 7, 6, 0, 1, 8, 5, 9, 0, 3],
        [3, 2, 0, 4, 2, 7, 2, 2, 6, 3, 2, 3, 5]]),
 array([[2, 1, 3, 5, 5, 6, 7, 7, 8, 4, 1, 9, 8],
        [5, 9, 5, 2, 0, 9, 2, 1, 9, 7, 9, 7, 6],
        [1, 7, 4, 0, 0, 7, 2, 2, 4, 1, 6, 4, 0],
        [6, 8, 1, 8, 7, 6, 0, 1, 8, 5, 9, 0, 3],
        [3, 2, 0, 4, 2, 7, 2, 2, 6, 3, 2, 3, 5]]),
 array([ 3, 28,  8, 29, 28, 25,  0,  7, 25, 14,  1,  6, 13]),
 array([ 3, 28,  8, 29, 28, 25,  0,  7, 25, 14,  1,  6, 13]))

In [159]:
xx,yy,xxx,yyy

(array([[2, 1, 3, 5, 5, 6, 7, 7, 8, 4, 1, 9, 8],
        [5, 9, 5, 2, 0, 9, 2, 1, 9, 7, 9, 7, 6],
        [1, 7, 4, 0, 0, 7, 2, 2, 4, 1, 6, 4, 0],
        [6, 8, 1, 8, 7, 6, 0, 1, 8, 5, 9, 0, 3],
        [3, 2, 0, 4, 2, 7, 2, 2, 6, 3, 2, 3, 5]]),
 array([ 3, 28,  8, 29, 28, 25,  0,  7, 25, 14,  1,  6, 13]),
 array([[8, 4, 1, 9, 8, 2, 1, 3, 7, 5, 5, 6, 7],
        [9, 7, 9, 7, 6, 5, 9, 5, 1, 2, 0, 9, 2],
        [4, 1, 6, 4, 0, 1, 7, 4, 2, 0, 0, 7, 2],
        [8, 5, 9, 0, 3, 6, 8, 1, 1, 8, 7, 6, 0],
        [6, 3, 2, 3, 5, 3, 2, 0, 2, 4, 2, 7, 2]]),
 array([25, 14,  1,  6, 13,  3, 28,  8,  7, 29, 28, 25,  0]))

In [189]:
pp = [3,4,1,5]
pp

[3, 4, 1, 5]

In [157]:
pr = [x for x in range(len(pp))]
random.Random(1).shuffle(pr)
pp,pr

(array([3, 4, 1, 5]), [3, 0, 2, 1])

In [158]:
s = 0
for i,x in enumerate(pr):
    xxx[:,s:s+pp[x]] = xx[:,sum(pp[:x]):sum(pp[:x])+pp[x]]
    yyy[s:s+pp[x]] = yy[sum(pp[:x]):sum(pp[:x])+pp[x]]
    s = s+pp[x]

In [192]:
[pp[pr[x]] for x in range(len(pp))]

[5, 3, 1, 4]

In [ ]:
combinations(dd.train_parts)

In [4]:
class DataMerge():
    def __init__(self,split = 0):
        self.split = split
        self.x_train = self.y_train = self.y_domain = self.train_parts = None
        self.x_val = self.y_val = self.y_valdom = self.val_parts = None
    def merge(self,data,train_test):
        if(train_test):
            print("HMM")
            if(self.x_val is None):self.x_val = data.trainX;
            else:self.x_val = np.concatenate((self.x_val,data.trainX),axis = 1)
            if(self.y_val is None):self.y_val = data.trainY
            else:self.y_val = np.concatenate((self.y_val,data.trainY),axis = 0)
            if(self.y_valdom is None):self.y_valdom = data.domainY
            else:self.y_valdom = self.y_valdom+data.domainY
            if(self.val_parts is None):
                if(data.train_parts is not None):self.val_parts = data.train_parts
            else:
                if(data.train_parts is not None):
                    self.val_parts = np.concatenate((self.val_parts,data.train_parts),axis = 0)
                else:
                    print("Data train parts unavailable")
            if(self.split>0):        
                if(self.x_train is None):self.x_train = data.valX
                else:self.x_train = np.concatenate((self.x_train,data.valX),axis = 1)
                if(self.y_train is None):self.y_train =  data.valY
                else:self.y_train = np.concatenate((self.y_train,data.valY),axis = 0)
                if(self.y_domain is None):self.y_domain  =  data.valdomY
                else:self.y_domain = self.y_domain+data.valdomY
                if(self.train_parts is None):self.train_parts = data.val_parts;
                else:
                    if(data.val_parts is not None):
                        self.train_parts = np.concatenate((self.train_parts,data.val_parts),axis = 0) 
                    else:
                        print("Data train parts unavailable")

        else:
            
            if(self.x_train is None):self.x_train = data.trainX;
            else:self.x_train = np.concatenate((self.x_train,data.trainX),axis = 1)
            if(self.y_train is None):self.y_train = data.trainY;
            else:self.y_train = np.concatenate((self.y_train,data.trainY),axis = 0)
            if(self.y_domain is None):self.y_domain = data.domainY;
            else:self.y_domain = self.y_domain+data.domainY
            if(self.train_parts is None):
                self.train_parts = data.train_parts;
            else:
                if(data.train_parts is not None):
                    self.train_parts = np.concatenate((self.train_parts,data.train_parts),axis = 0) 
                else:
                    print("Data train parts nai Train mergee ")

def getData(fold_dir, train_folds, test_folds, split = 0):
    try:
        with open('../data/domain_filename.json', 'r') as fp:
            foldname = json.load(fp)
    except:
        raise FileNotFoundError("The json file that maps domain character to filename is not here")

    allData = DataMerge(split)
    for c in test_folds:
        allData.merge(Data(fold_dir,foldname[c],c,severe = False,split=split),True)
    for c in train_folds:
        allData.merge(Data(fold_dir,foldname[c],c),False)
        
    return allData.x_train, allData.y_train, allData.y_domain, allData.train_parts,allData.x_val,allData.y_val,allData.y_valdom,allData.val_parts 


def reshape_folds(x, y):
    x_train = []
    for x1 in x:
        x1 = np.transpose(x1[:, :])
        x1 = np.reshape(x1, [x1.shape[0], x1.shape[1], 1])
        x_train.append(x1)
        print("reshaped x ", x1.shape)
    y_train = []
    for y1 in y:
        y1 = np.reshape(y1, [y1.shape[0], 1])
        y_train.append(y1)
        print("reshaped Y ", y1.shape)

    return x_train,y_train

In [5]:
hm = getData(fold_dir,'abcdefg','hi')

../../feature/potes_1DCNN/balancedCV/folds/all_folds_wav_name/pascalB.mat
True
HMM
../../feature/potes_1DCNN/balancedCV/folds/all_folds_wav_name/compare.mat
True
HMM
../../feature/potes_1DCNN/balancedCV/folds/all_folds_wav_name/fold_a.mat
True
../../feature/potes_1DCNN/balancedCV/folds/all_folds_wav_name/fold_b.mat
True
../../feature/potes_1DCNN/balancedCV/folds/all_folds_wav_name/fold_c.mat
True
../../feature/potes_1DCNN/balancedCV/folds/all_folds_wav_name/fold_d.mat
True
../../feature/potes_1DCNN/balancedCV/folds/all_folds_wav_name/fold_e.mat
True
../../feature/potes_1DCNN/balancedCV/folds/all_folds_wav_name/fold_f.mat
True
../../feature/potes_1DCNN/balancedCV/folds/all_folds_wav_name/pascalA.mat
True


In [ ]:
os.path.isfile

In [20]:
u,i,o,p,uu,ii,oo,pp = getData(fold_dir,'abcdefgh','i',split = 0.0)

testc i
HMM
train  a
train  b
train  c
train  d
train  e
train  f
train  g
train  h


In [130]:
u.shape, uu.shape

((2500, 20275), (2500, 4159))

In [133]:
u.shape, uu.shape

((2500, 19734), (2500, 4700))

In [15]:
fails = ['fold_a.mat',
 'fold_b.mat',
 'fold_c.mat',
 'fold_d.mat',
 'fold_e.mat',
 'fold_f.mat',
 'pascalA.mat',
 'pascalB.mat',
 'compare.mat']

In [18]:
for (x,n) in zip(fails,'abcdefghi'):
    d = Data(fold_dir,x,n,split=0.0)
    print(x)
    d.details()
    d.types()

fold_a.mat
valX/ valY/ Valdom/ val parts/ 
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'list'> <class 'numpy.ndarray'>
fold_b.mat
valX/ valY/ Valdom/ val parts/ 
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'list'> <class 'numpy.ndarray'>
fold_c.mat
valX/ valY/ Valdom/ val parts/ 
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'list'> <class 'numpy.ndarray'>
fold_d.mat
valX/ valY/ Valdom/ val parts/ 
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'list'> <class 'numpy.ndarray'>
fold_e.mat
valX/ valY/ Valdom/ val parts/ 
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'list'> <class 'list'>
fold_f.mat
valX/ valY/ Valdom/ val parts/ 
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'list'> <class 'numpy.ndarray'>
pascalA.mat
valX/ valY/ Valdom/ val parts/ 
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class 'list'> <class 'numpy.ndarray'>
pascalB.mat
valX/ valY/ Valdom/ val parts/ 
<class 'numpy.ndarray'> <class 'numpy.ndarray'> <class

In [98]:
ww = [2,2,2,2]
all([x==2 for x in ww])

True

In [101]:
'fold_e' in "fold_e_dfasfd"

True